# Preliminary examination of logged Generation warnings from SSEN website 

## load sql extension and connect to database file

In [2]:
## Make copy of original databse for examination and cleaning

from shutil import copyfile

copyfile('../logging/database/ANMlog.db','./database/ANMlog_copy.db')

'./database/ANMlog_copy.db'

In [1]:
%load_ext sql

In [3]:
%sql sqlite:///./database/ANMlog_copy.db

'Connected: @./database/ANMlog_copy.db'

In [4]:
%sql SELECT COUNT(*) FROM records;

 * sqlite:///./database/ANMlog_copy.db
Done.


COUNT(*)
15714


## Get some basic information about the database

In [5]:
%sql pragma table_info('records')

 * sqlite:///./database/ANMlog_copy.db
Done.


cid,name,type,notnull,dflt_value,pk
0,timestamp,DATETIME,0,None,0
1,log,TEXT,0,None,0


In [6]:
%sql SELECT min(timestamp), max(timestamp) FROM records;

 * sqlite:///./database/ANMlog_copy.db
Done.


min(timestamp),max(timestamp)
2019-01-16 22:42:33,2019-10-27 13:35:16


In [7]:
%sql SELECT * from records limit 10;

 * sqlite:///./database/ANMlog_copy.db
Done.


timestamp,log
2019-01-16 22:42:33,"{""2"": [""warning"", ""ok"", ""ok""], ""2B"": [""warning"", ""ok"", ""ok""], ""2A"": [""remove"", ""ok"", ""ok""], ""4A"": [""ok"", ""ok"", ""ok""], ""3"": [""ok"", ""ok"", ""ok""], ""Core"": [""ok"", ""ok"", ""ok""], ""1"": [""ok"", ""ok"", ""ok""], ""4"": [""ok"", ""ok"", ""ok""], ""1A"": [""ok"", ""ok"", ""ok""]}"
2019-01-16 22:46:25,"{""1"": [""ok"", ""ok"", ""ok""], ""1A"": [""ok"", ""ok"", ""ok""], ""2"": [""warning"", ""ok"", ""ok""], ""2A"": [""remove"", ""ok"", ""ok""], ""2B"": [""warning"", ""ok"", ""ok""], ""3"": [""ok"", ""ok"", ""ok""], ""4"": [""ok"", ""ok"", ""ok""], ""4A"": [""ok"", ""ok"", ""ok""], ""Core"": [""ok"", ""ok"", ""ok""]}"
2019-01-16 22:59:22,"{""1"": [""ok"", ""ok"", ""ok""], ""1A"": [""ok"", ""ok"", ""ok""], ""2"": [""ok"", ""ok"", ""ok""], ""2A"": [""remove"", ""ok"", ""ok""], ""2B"": [""warning"", ""ok"", ""ok""], ""3"": [""ok"", ""ok"", ""ok""], ""4"": [""ok"", ""ok"", ""ok""], ""4A"": [""ok"", ""ok"", ""ok""], ""Core"": [""ok"", ""ok"", ""ok""]}"
2019-01-16 23:03:05,"{""1"": [""ok"", ""ok"", ""ok""], ""1A"": [""ok"", ""ok"", ""ok""], ""2"": [""ok"", ""ok"", ""ok""], ""2A"": [""remove"", ""ok"", ""ok""], ""2B"": [""warning"", ""ok"", ""ok""], ""3"": [""ok"", ""ok"", ""ok""], ""4"": [""ok"", ""ok"", ""ok""], ""4A"": [""ok"", ""ok"", ""ok""], ""Core"": [""ok"", ""ok"", ""ok""]}"
2019-01-16 23:06:26,"{""1"": [""ok"", ""ok"", ""ok""], ""1A"": [""ok"", ""ok"", ""ok""], ""2"": [""warning"", ""ok"", ""ok""], ""2A"": [""remove"", ""ok"", ""ok""], ""2B"": [""warning"", ""ok"", ""ok""], ""3"": [""ok"", ""ok"", ""ok""], ""4"": [""ok"", ""ok"", ""ok""], ""4A"": [""ok"", ""ok"", ""ok""], ""Core"": [""ok"", ""ok"", ""ok""]}"
2019-01-17 09:18:10,"{""1"": [""ok"", ""ok"", ""ok""], ""1A"": [""ok"", ""ok"", ""ok""], ""2"": [""ok"", ""ok"", ""ok""], ""2A"": [""warning"", ""ok"", ""ok""], ""2B"": [""ok"", ""ok"", ""ok""], ""3"": [""ok"", ""ok"", ""ok""], ""4"": [""ok"", ""ok"", ""ok""], ""4A"": [""ok"", ""ok"", ""ok""], ""Core"": [""ok"", ""ok"", ""ok""]}"
2019-01-17 09:19:10,"{""1"": [""ok"", ""ok"", ""ok""], ""1A"": [""ok"", ""ok"", ""ok""], ""2"": [""ok"", ""ok"", ""ok""], ""2A"": [""ok"", ""ok"", ""ok""], ""2B"": [""ok"", ""ok"", ""ok""], ""3"": [""ok"", ""ok"", ""ok""], ""4"": [""ok"", ""ok"", ""ok""], ""4A"": [""ok"", ""ok"", ""ok""], ""Core"": [""ok"", ""ok"", ""ok""]}"
2019-01-17 09:21:11,"{""1"": [""ok"", ""ok"", ""ok""], ""1A"": [""ok"", ""ok"", ""ok""], ""2"": [""warning"", ""ok"", ""ok""], ""2A"": [""ok"", ""ok"", ""ok""], ""2B"": [""ok"", ""ok"", ""ok""], ""3"": [""ok"", ""ok"", ""ok""], ""4"": [""ok"", ""ok"", ""ok""], ""4A"": [""ok"", ""ok"", ""ok""], ""Core"": [""ok"", ""ok"", ""ok""]}"
2019-01-17 09:27:10,"{""1"": [""ok"", ""ok"", ""ok""], ""1A"": [""ok"", ""ok"", ""ok""], ""2"": [""remove"", ""ok"", ""ok""], ""2A"": [""remove"", ""ok"", ""ok""], ""2B"": [""warning"", ""ok"", ""ok""], ""3"": [""ok"", ""ok"", ""ok""], ""4"": [""ok"", ""ok"", ""ok""], ""4A"": [""ok"", ""ok"", ""ok""], ""Core"": [""ok"", ""ok"", ""ok""]}"
2019-01-17 09:29:11,"{""1"": [""ok"", ""ok"", ""ok""], ""1A"": [""ok"", ""ok"", ""ok""], ""2"": [""warning"", ""ok"", ""ok""], ""2A"": [""remove"", ""ok"", ""ok""], ""2B"": [""warning"", ""ok"", ""ok""], ""3"": [""ok"", ""ok"", ""ok""], ""4"": [""ok"", ""ok"", ""ok""], ""4A"": [""ok"", ""ok"", ""ok""], ""Core"": [""ok"", ""ok"", ""ok""]}"


## Search for duplicate entries

In [8]:
%%sql 
SELECT timestamp, COUNT(*) count 
FROM records 
GROUP BY timestamp 
HAVING count > 1;

 * sqlite:///./database/ANMlog_copy.db
Done.


timestamp,count


## Search for multiple readings within single minute periods 

In [9]:
%%sql 
SELECT substr(timestamp, 0, 17) as timestamp, COUNT(*) as count 
FROM records 
GROUP BY substr(timestamp, 0, 17) 
HAVING count > 1;

 * sqlite:///./database/ANMlog_copy.db
Done.


timestamp,count
2019-01-21 20:58,2
2019-05-31 21:54,2
2019-05-31 22:00,2


count of multiple entries within any single minute period

In [10]:
%sql SELECT sum(count) from (SELECT substr(timestamp, 0, 17) as timestamp, COUNT(*) as count FROM records GROUP BY substr(timestamp, 0, 17) HAVING count > 1);

 * sqlite:///./database/ANMlog_copy.db
Done.


sum(count)
6


## count of minute periods with multiple entries

In [11]:
%%sql
SELECT count(*) 
FROM (SELECT substr(timestamp, 0, 17) as timestamp, COUNT(*) as count 
      FROM records
      GROUP BY substr(timestamp, 0, 17) 
      HAVING count > 1
     )
;

 * sqlite:///./database/ANMlog_copy.db
Done.


count(*)
3


In [12]:
%%sql
SELECT timestamp 
FROM records 
WHERE substr(timestamp, 0, 17) IN 
(SELECT timestamp 
 FROM (SELECT substr(timestamp, 0, 17) as timestamp, COUNT(*) c 
       FROM records 
       GROUP BY substr(timestamp, 0, 17) 
       HAVING c > 1
      )
);

 * sqlite:///./database/ANMlog_copy.db
Done.


timestamp
2019-01-21 20:58:10
2019-01-21 20:58:57
2019-05-31 21:54:17
2019-05-31 21:54:20
2019-05-31 22:00:28
2019-05-31 22:00:38


## get readings with row id numbers


In [13]:
%%sql SELECT ROWID, substr(timestamp, 0, 17) as datetime, substr(timestamp, 18, 19) as seconds 
FROM records 
WHERE substr(timestamp, 0, 17)
IN (SELECT timestamp 
    FROM (SELECT substr(timestamp, 0, 17) as timestamp, COUNT(*) c  
          FROM records
          GROUP BY substr(timestamp, 0, 17)
          HAVING c > 1
         )
   )
;

 * sqlite:///./database/ANMlog_copy.db
Done.


rowid,datetime,seconds
223,2019-01-21 20:58,10
224,2019-01-21 20:58,57
10512,2019-05-31 21:54,17
10513,2019-05-31 21:54,20
10515,2019-05-31 22:00,28
10516,2019-05-31 22:00,38


so rows 224, 10513, 10516 have been identified as extraneous and can be deleted.  

In [14]:
%%sql
select rowid, timestamp from records where rowid in (224, 10513, 10516);

 * sqlite:///./database/ANMlog_copy.db
Done.


rowid,timestamp
224,2019-01-21 20:58:57
10513,2019-05-31 21:54:20
10516,2019-05-31 22:00:38


In [15]:
%%sql
delete from records where rowid in (224, 10513, 10516);

 * sqlite:///./database/ANMlog_copy.db
3 rows affected.


[]

In [16]:
%%sql
SELECT ROWID, timestamp 
FROM records 
WHERE substr(timestamp, 0, 17) IN 
(SELECT timestamp 
 FROM (SELECT substr(timestamp, 0, 17) as timestamp, COUNT(*) c 
       FROM records 
       GROUP BY substr(timestamp, 0, 17) 
       HAVING c > 1
      )
);

 * sqlite:///./database/ANMlog_copy.db
Done.


rowid,timestamp


# convert json data to sql

In [17]:
import json

In [18]:
data = %sql SELECT RowId as id, * FROM records;

 * sqlite:///./database/ANMlog_copy.db
Done.


In [19]:
data[222]

(223, '2019-01-21 20:58:10', '{"1": ["ok", "ok", "ok"], "1A": ["ok", "ok", "ok"], "2": ["ok", "ok", "ok"], "2A": ["ok", "ok", "ok"], "2B": ["remove", "ok", "ok"], "3": ["ok", "ok", "ok"], "4": ["ok", "ok", "ok"], "4A": ["ok", "ok", "ok"], "Core": ["ok", "ok", "ok"]}')

In [20]:
anm = json.loads(data[0]['log'])
anm

{'2': ['warning', 'ok', 'ok'],
 '2B': ['warning', 'ok', 'ok'],
 '2A': ['remove', 'ok', 'ok'],
 '4A': ['ok', 'ok', 'ok'],
 '3': ['ok', 'ok', 'ok'],
 'Core': ['ok', 'ok', 'ok'],
 '1': ['ok', 'ok', 'ok'],
 '4': ['ok', 'ok', 'ok'],
 '1A': ['ok', 'ok', 'ok']}

In [21]:
anm['2']

['warning', 'ok', 'ok']

In [22]:
anm['2'][0]

'warning'